# 布林带策略

In [1]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time

# 回测框架
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeed

# 基础函数
import utilsJ

## 策略主体

买入信号：收盘价向上突破布林带上界或下界时买入。

买入仓位：每次满仓买入(做空同理)。

卖出信号：收盘价向下突破布林带上界或下界时卖出。

卖出仓位：清仓。

### 个股版本

In [2]:
class Bollinger_s(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('allow_short', False),
        ('stake', 100),

        # Indicator params
        ('bbands_per', 20),
        ('bbands_dev', 2),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        if self.p.printlog or doprint:
            dt = dt or self.data.datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        # Initialization
        self.order = None
        self.buyprice = None
        self.sellprice = None

        # Alias
        self.dataclose = self.data.close

        # Indicators
        ## Bollinger
        self.bbands = btind.BBands(self.dataclose,
                                   period=self.p.bbands_per, 
                                   devfactor=self.p.bbands_dev)

        # Signals
        self.buy_s = bt.Or(bt.And(self.dataclose > self.bbands.top, 
                                  self.dataclose(-1) < self.bbands.top(-1)),
                           bt.And(self.dataclose > self.bbands.bot, 
                                  self.dataclose(-1) < self.bbands.bot(-1)))
        
        self.sell_s = bt.Or(bt.And(self.dataclose < self.bbands.top, 
                                   self.dataclose(-1) > self.bbands.top(-1)),
                            bt.And(self.dataclose < self.bbands.bot, 
                                   self.dataclose(-1) > self.bbands.bot(-1)))


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                         (order.executed.price, order.executed.size,
                          self.broker.get_cash(), self.broker.get_value()))
                self.buyprice = order.executed.price

            else:
                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                        (order.executed.price, -order.executed.size,
                         self.broker.get_cash(), self.broker.get_value()))
                self.sellprice = order.executed.price

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):
        if self.order:
            return

        ava_pos = self.broker.get_cash() // (self.dataclose[0] * self.p.stake) * self.p.stake

        if self.buy_s[0]:
            if self.getposition(self.data).size == 0 : # Open position
                self.log('BUY CREATE, Price: %.2f, Lots: %i' %
                         (self.dataclose[0], ava_pos))
                self.order = self.buy(size=ava_pos)
            elif self.getposition(self.data).size < 0: # Close position
                self.log('BUY CREATE (Close), Price: %.2f, Lots: %i' %
                         (self.dataclose[0], -self.getposition(self.data).size))
                self.order = self.close()

        elif self.sell_s[0]:
            if self.getposition(self.data).size == 0 and self.p.allow_short: # Open position
                self.log('SELL CREATE, Price: %.2f, Lots: %i' %
                         (self.dataclose[0], ava_pos))
                self.order = self.sell(size=ava_pos)
            elif self.getposition(self.data).size > 0: # Close position
                self.log('SELL CREATE (Close), Price: %.2f, Lots: %i' %
                         (self.dataclose[0], self.getposition(self.data).size))
                self.order = self.close()

## 回测

### 个股回测

In [3]:
stock_code = '002057.SZ'
startdate = dt.datetime(2021,1,1)
enddate = dt.datetime(2021,12,31)

if __name__ ==  '__main__':
    
    # Initialization
    cerebro = bt.Cerebro()
    strats = cerebro.addstrategy(Bollinger_s, printlog=True) 

    # Data
    df = utilsJ.get_stock(stock_code, startdate, enddate)
    data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)
    cerebro.adddata(data)
    
    # Start condition
    cerebro.broker = bt.brokers.BackBroker(coc=True)
    cerebro.broker.setcash(10000)
    #cerebro.broker.setcommission()
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Execution
    cerebro.run()

    # Final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.plot()

Starting Portfolio Value: 10000.00
2021-03-02: BUY CREATE, Price: 7.15, Lots: 1300
2021-03-03: BUY EXECUTED, Price: 7.15, Lot:1300, Cash: 707, Value: 10429
2021-03-05: SELL CREATE (Close), Price: 7.48, Lots: 1300
2021-03-08: SELL EXECUTED, Price: 7.48, Lot:1300, Cash: 10429, Value: 10429
2021-03-08: OPERATION PROFIT, GROSS 429.26, NET 429.26
2021-04-02: BUY CREATE, Price: 6.68, Lots: 1500
2021-04-06: BUY EXECUTED, Price: 6.68, Lot:1500, Cash: 406, Value: 10676
2021-06-01: SELL CREATE (Close), Price: 6.76, Lots: 1500
2021-06-02: SELL EXECUTED, Price: 6.76, Lot:1500, Cash: 10545, Value: 10545
2021-06-02: OPERATION PROFIT, GROSS 116.55, NET 116.55
2021-06-07: BUY CREATE, Price: 6.98, Lots: 1500
2021-06-08: BUY EXECUTED, Price: 6.98, Lot:1500, Cash: 71, Value: 10472
2021-06-10: SELL CREATE (Close), Price: 6.98, Lots: 1500
2021-06-11: SELL EXECUTED, Price: 6.98, Lot:1500, Cash: 10545, Value: 10545
2021-06-11: OPERATION PROFIT, GROSS 0.00, NET 0.00
2021-07-07: BUY CREATE, Price: 7.08, Lots: 

### 股指期货回测调参 (沪深当月)

In [5]:
future_code = 'IFL.CFX'
startdate = dt.datetime(2021,1,1)
enddate = dt.datetime(2021,12,31)

if __name__ ==  '__main__':
    
    df = utilsJ.future_ts(future_code, startdate, enddate)
    max_np = 0.0
    max_idx = (0, 0.0)
    
    for i in range(3, 31, 1):
        for j in np.arange(0.2, 2.1, 0.1):
            
            # Initialization
            cerebro = bt.Cerebro()
            strats = cerebro.addstrategy(Bollinger_s, printlog=False, bollinger_per=i, bollinger_dev=float(j), allow_short=True)

            # Data
            data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)
            cerebro.adddata(data)

            # Start condition
            cerebro.broker = bt.brokers.BackBroker(coc=True)
            cerebro.broker.setcash(100000.00)
            start_value = cerebro.broker.getvalue()
            #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

            # Execution
            cerebro.run()

            # Final results
            final_value = cerebro.broker.getvalue()
            netp = (final_value / start_value - 1) * 100
            if netp > max_np:
                max_np = netp
                max_idx = (i, j)
    print(max_np, max_idx)

0.0 (0, 0.0)
